In [1]:
import pandas as pandas
import nltk
from nltk.corpus import stopwords
import time
import re
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer 

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [2]:
import gensim

In [3]:
filename = 'train_data.csv'
t1 = time.time()
data = pandas.read_csv('train_data.csv')

In [4]:
data.head()

,stars,name,text,date,city,longitude,latitude,categories
0,1,McDonald's,Seriously cannot stand this McDonald's. They N...,2014-12-29,Glendale,-112.205020,33.509597,"['Burgers', 'Fast Food', 'Restaurants']"
1,5,Tom Colicchio's Craftsteak,"Amazing food, truly excellent best lobster bis...",2013-03-07,Las Vegas,-115.169751,36.102918,"['Steakhouses', 'Restaurants', 'Cheesesteaks',..."
2,5,Fishman Lobster Clubhouse Restaurant,"This was my second time here, and the seafood ...",2015-11-24,Toronto,-79.300795,43.824234,"['Seafood', 'Restaurants', 'Chinese', 'Live/Ra..."
3,1,Bonjour Brioche,"Long story short.\n\nBunch of rude, heartless,...",2016-12-20,Toronto,-79.346287,43.659795,"['Breakfast & Brunch', 'French', 'Restaurants']"
4,4,Dilly's Deli,We grabbed some dinner here last night before ...,2010-09-28,Tempe,-111.945365,33.422175,"['Caterers', 'Sandwiches', 'Event Planning & S..."


In [10]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=3, scheme='http') 
geolocator = geopy.geocoders.GoogleV3(timeout=3
location = geolocator.reverse("52.5, 13.37")
#print(location.address)

GeocoderInsufficientPrivileges: HTTP Error 403: Forbidden

In [11]:
data_5=data[data["stars"]==5]
data_txt5=data_5["text"]
data_txt5=data_txt5[0:50]
data_txt5.to_csv("5 star.csv",sep=",")

data_1=data[data["stars"]==1]
data_txt1=data_1["text"]
data_txt1=data_txt1[3:53]
data_txt1.to_csv("1 star.csv",sep=",")

In [12]:
#nltk.download()

In [33]:
nreviews=1000
#nreviews=data.shape[0]
stop = set(stopwords.words("english"))

data_modeltxt=data["text"]
data_modeltxt=data_modeltxt[0:nreviews]
data_modeltxt.head()
    
def stop_delete(sentence):
    sentence=sentence.lower().split()
    sen=[word for word in sentence if not word in stop]
    sen_re=np.array([])
    for j in sen:
        word_re=re.sub('[^a-zA-Z]','', j)
        if word_re != "":
            sen_re=np.append(sen_re,word_re)
    return sen_re

In [34]:
from gensim import corpora
from collections import defaultdict

lem = WordNetLemmatizer()
stem = PorterStemmer()

document=[];
for i in data_modeltxt:
    txt=stop_delete(i)
    txt=[stem.stem(word) for word in txt]
    document.append(txt);

# remove words that appear only once
frequency=defaultdict(int)
for txt in document:
    for word in txt:
        frequency[word] += 1

document = [[word for word in txt if frequency[word] > 1] for txt in document]

In [48]:
import csv

with open('document_txt.csv', 'w', newline="") as myfile:
    writer = csv.writer(myfile)
    writer.writerows(document)

with open('frequency_txt.csv', 'w', newline="") as myfile:
    writer = csv.writer(myfile)
    for key, value in frequency.items():
        writer.writerow([key, value])

In [51]:
with open("document_txt.csv", 'r') as filein: 
    reader = csv.reader(filein)
    document = list(list(rec) for rec in csv.reader(filein, delimiter=',')) 
    
with open("frequency_txt.csv", 'r') as filein: 
    reader = csv.reader(filein)
    frequency = dict(reader)

In [52]:
#dictionary
dictionary = corpora.Dictionary(document)
dictionary.save('yelpword.dict')
#list to corpus
corpus = [dictionary.doc2bow(txt) for txt in document]
corpora.MmCorpus.serialize('yelpreview.mm', corpus)
document = corpora.MmCorpus('yelpreview.mm')

In [53]:
LDA = gensim.models.ldamodel.LdaModel(corpus=document, id2word=dictionary, num_topics=100)

In [54]:
LDA.print_topics(5)

[(50,
  '0.017*"good" + 0.014*"bar" + 0.013*"wir" + 0.013*"new" + 0.013*"chico" + 0.011*"coffe" + 0.011*"place" + 0.011*"get" + 0.010*"die" + 0.010*"food"'),
 (41,
  '0.015*"good" + 0.012*"order" + 0.012*"food" + 0.012*"pizza" + 0.011*"place" + 0.010*"got" + 0.010*"would" + 0.009*"it" + 0.009*"friend" + 0.008*"tri"'),
 (84,
  '0.017*"place" + 0.015*"good" + 0.012*"servic" + 0.010*"food" + 0.009*"great" + 0.009*"time" + 0.008*"best" + 0.008*"well" + 0.007*"like" + 0.007*"order"'),
 (37,
  '0.019*"good" + 0.011*"drink" + 0.010*"soup" + 0.010*"realli" + 0.010*"order" + 0.009*"eat" + 0.009*"servic" + 0.009*"much" + 0.009*"bar" + 0.008*"place"'),
 (14,
  '0.020*"food" + 0.015*"good" + 0.014*"burger" + 0.013*"great" + 0.011*"get" + 0.009*"littl" + 0.008*"order" + 0.008*"place" + 0.007*"chees" + 0.007*"servic"')]